In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry
import psycopg2
from sqlalchemy import create_engine
import geoalchemy2
import requests
import zipfile

# Set working directory
working_directory = r"C:\Users\benla\Desktop\Grad_School\Classes\GIS5572_SpatialDataScience\Assignments\Group_Project\Data"
os.chdir(working_directory)

In [2]:
#database connection parameters
db_params = {
    "host": "34.30.71.239",
    "port": "5432",
    "dbname": "gis_data",
    "user": "postgres",
    "password": ""
}

#db connection string
db_string = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'
engine = create_engine(db_string)

In [3]:
#define cooling center table variable and load

cooling_options = "cooling_options"
cooling_options_df = pd.read_sql(f"SELECT * FROM {cooling_options}", engine)

In [4]:
#visualize table
print(cooling_options_df)

      id                                           geometry  \
0      1  010100002023690000D03B4ED151651B4152499D9CCA06...   
1      2  010100002023690000F0E461E1F6661B412041F18B9A06...   
2      3  010100002023690000D0E53F64C4221B41A067B31610FC...   
3      4  0101000020236900003080B780EE371B410EE02DC854FC...   
4      5  010100002023690000D076BEDF375F1B41E09C11C506FD...   
..   ...                                                ...   
313  314  010100002023690000E03FA41F58D21C415A643BBF1302...   
314  315  01010000202369000080C954011DCA1C419EEFA7861303...   
315  316  010100002023690000E0240681FBB91C41EE7C3F9DC2FA...   
316  317  010100002023690000D07F48BFB2B31C416ABC74BB1A0C...   
317  318  010100002023690000D012F24126151D417C3F35FE91FC...   

                                                name                address  \
0                                    Camp Ihduhapi 1    3425 Ihduhapi Trail   
1                                    Camp Ihduhapi 2    3425 Ihduhapi Trail   
2     

In [5]:
#view specifically 'fee' and 'inhennepin' fields
print(cooling_options_df[['fee','inhennepin']])

     fee inhennepin
0    Yes   Hennepin
1    Yes   Hennepin
2    Yes   Hennepin
3    Yes   Hennepin
4     No   Hennepin
..   ...        ...
313   No   Hennepin
314   No   Hennepin
315   No   Hennepin
316  Yes   Hennepin
317   No   Hennepin

[318 rows x 2 columns]


In [6]:
#remove any cooling center where fee attribute = yes and inhennepin = no
cooling_options_df_filtered = cooling_options_df[(cooling_options_df["inhennepin"] != "No") & (cooling_options_df["fee"] != "Yes")]

In [9]:
#visualize filtered table
print(cooling_options_df_filtered)

      id                                           geometry  \
4      5  010100002023690000D076BEDF375F1B41E09C11C506FD...   
5      6  010100002023690000806132D560891B41A2B4377C41FA...   
6      7  010100002023690000E02D90A0E05A1B41984C150CD0F9...   
7      8  010100002023690000C07493D84EE11B41A679C7CD21FA...   
8      9  010100002023690000E08C28ADE79A1B41CEAACFCD80FA...   
..   ...                                                ...   
312  313  010100002023690000B0A44EC0BF4C1D4188D2DE8C90FC...   
313  314  010100002023690000E03FA41F58D21C415A643BBF1302...   
314  315  01010000202369000080C954011DCA1C419EEFA7861303...   
315  316  010100002023690000E0240681FBB91C41EE7C3F9DC2FA...   
317  318  010100002023690000D012F24126151D417C3F35FE91FC...   

                                                name                address  \
4                                       Canary Beach        1550 Canary Ln.   
5                                  Casco Point Beach  2879 Casco Point Road   
6     

In [10]:
#double check filtered columns
print(cooling_options_df_filtered[['fee','inhennepin']])

    fee inhennepin
4    No   Hennepin
5    No   Hennepin
6    No   Hennepin
7    No   Hennepin
8    No   Hennepin
..   ..        ...
312  No   Hennepin
313  No   Hennepin
314  No   Hennepin
315  No   Hennepin
317  No   Hennepin

[250 rows x 2 columns]


In [17]:
#make the cooling options dataframe a geodataframe with geopandas
cooling_options_gdf = gpd.GeoDataFrame(cooling_options_df_filtered, geometry=gpd.points_from_xy(cooling_options_df_filtered["longitude"], cooling_options_df_filtered["latitude"]))

cooling_options_gdf=cooling_options_gdf[cooling_options_gdf.latitude!=0]

cooling_options_gdf

,id,geometry,name,address,city,zip,phone,type,status,hours,...,in_out,notes,joinfield,utm_x,utm_y,latitude,longitude,climateana,censustrac,inhennepin
4,5,POINT (-93.65335 44.95079),Canary Beach,1550 Canary Ln.,Mound,55364,None,HC Beach,Active,None,...,Outdoor,None,0,448461.9685,4.977691e+06,44.950787,-93.653350,30.699513,27053027601,Hennepin
5,6,POINT (-93.61887 44.92544),Casco Point Beach,2879 Casco Point Road,Wayzata,55391,None,HC Beach,Active,None,...,Outdoor,None,0,451160.2082,4.974854e+06,44.925439,-93.618872,38.110868,27053027203,Hennepin
6,7,POINT (-93.65653 44.92114),Chester Beach,5186 Tuxedo Blvd,Mound,55364,None,HC Beach,Active,None,...,Outdoor,None,0,448184.1568,4.974400e+06,44.921144,-93.656534,22.223379,27053027602,Hennepin
7,8,POINT (-93.54756 44.92466),Linwood Beach,20700 Linwood Road,Excelsior,55331,None,HC Beach,Active,None,...,Outdoor,None,0,456787.7115,4.974727e+06,44.924662,-93.547556,11.712170,27053027400,Hennepin
8,9,POINT (-93.60468 44.92779),Lydiard Beach,Lydiard Avenue,Excelsior,55331,None,HC Beach,Active,None,...,Outdoor,None,0,452281.9191,4.975107e+06,44.927795,-93.604683,38.110868,27053027203,Hennepin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,308,POINT (-93.32624 44.8813),Southdale Shopping Center,6601 France Av S,Edina,55435,952-925-7874,Shopping Mall,Active,"Mon-Sat 10am-8pm, Sun 12pm-6pm",...,Indoor,None,74,474234.5278,4.969816e+06,44.881296,-93.326237,65.395251,27053024005,Hennepin
308,309,POINT (-93.32748 44.87717),Galleria,69th Street & France Ave,Edina,55435,952-925-4321,Shopping Mall,Active,"Mon-Sat 10am-7pm, Sun 11am-6pm",...,Indoor,None,75,474134.3458,4.969358e+06,44.877171,-93.327482,65.395251,27053024005,Hennepin
309,310,POINT (-93.26086 44.94853),Midtown Global Market,920 E. Lake St.,Minneapolis,55407,612-964-6932,Shopping Mall,Active,"Mon-Sat 10am-8pm, Sun 11am-6pm.",...,Indoor,Customers can park for free in the ramp. Indiv...,0,479421.8953,4.977265e+06,44.948527,-93.260859,70.764093,27053125800,Hennepin
311,312,POINT (-93.43296 44.8629),Asia Mall,12160 Technology Dr.,Eden Prairie,55344,None,Shopping Mall,Active,Open daily 10am-10pm.,...,Indoor,None,0,465794.9392,4.967812e+06,44.862905,-93.432959,58.738406,27053026019,Hennepin


In [19]:
#double check by verifying all cooling center points are with hennepin county polygon

#download mn counties shapefile
url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip"
response = requests.get(url)
with open("mn_county.zip", "wb") as f:
    f.write(response.content)

# Extract the GeoPackage file
with zipfile.ZipFile("mn_county.zip", "r") as zip_ref:
    zip_ref.extractall("mn_county")

In [21]:
#load the downloaded shapefile with geopandas and print to view
mn_county_path = r"C:\Users\benla\Desktop\Grad_School\Classes\GIS5572_SpatialDataScience\Assignments\Group_Project\Data\mn_county\mn_county_boundaries_500.shp"
mn_county = gpd.read_file(mn_county_path)

#only need county name
print(mn_county['CTY_NAME'])

0      Lake of the Woods
1                Kittson
2                 Roseau
3            Koochiching
4               Marshall
             ...        
112                 Cook
113                 Cook
114                 Cook
115                 Lake
116                 Lake
Name: CTY_NAME, Length: 117, dtype: object


In [23]:
#filter to just be hennepin county
hennepin_county = mn_county[(mn_county["CTY_NAME"] == "Hennepin")]
print(hennepin_county)
hennepin_county.explore()

            AREA      PERIMETER  CTYONLY_  CTYONLY_ID  COUN  CTY_NAME  \
49  1.570587e+09  190077.767485      51.0        50.0    27  Hennepin   

   CTY_ABBR  CTY_FIPS  MaxSimpTol  MinSimpTol     Shape_Leng    Shape_Area  \
49     HENN        53       500.0       500.0  176369.888097  1.569322e+09   

                                             geometry  
49  POLYGON ((459105.693 5010507.644, 459790.33 50...  


In [24]:
# Check CRS of cooling centers and hennepin county 
print(cooling_options_gdf.crs)  
print(hennepin_county.crs) 

None
EPSG:26915


In [27]:
#to check if all cooling centers are within hennepin county, first check both df using the same CRS

#set cooling_centers_gdf CRS
cooling_options_gdf = cooling_options_gdf.set_crs(epsg=4326,allow_override=True)  # Assign WGS 84 first
cooling_options_gdf = cooling_options_gdf.to_crs(epsg=26915)  # Convert to match Hennepin County

# hennepin_county = hennepin_county.to_crs(epsg=32615)  #UTM 15N
# cooling_options = cooling_options_gdf.to_crs(hennepin_county.crs)

In [29]:
print(cooling_options_gdf.crs) 
print(cooling_options_gdf.total_bounds)

EPSG:26915
[ 441226.45508825 4963496.6785263   485478.65317358 5006638.60079158]


In [31]:
cooling_options_gdf

,id,geometry,name,address,city,zip,phone,type,status,hours,...,in_out,notes,joinfield,utm_x,utm_y,latitude,longitude,climateana,censustrac,inhennepin
4,5,POINT (448461.968 4977691.079),Canary Beach,1550 Canary Ln.,Mound,55364,None,HC Beach,Active,None,...,Outdoor,None,0,448461.9685,4.977691e+06,44.950787,-93.653350,30.699513,27053027601,Hennepin
5,6,POINT (451160.209 4974853.941),Casco Point Beach,2879 Casco Point Road,Wayzata,55391,None,HC Beach,Active,None,...,Outdoor,None,0,451160.2082,4.974854e+06,44.925439,-93.618872,38.110868,27053027203,Hennepin
6,7,POINT (448184.157 4974400.189),Chester Beach,5186 Tuxedo Blvd,Mound,55364,None,HC Beach,Active,None,...,Outdoor,None,0,448184.1568,4.974400e+06,44.921144,-93.656534,22.223379,27053027602,Hennepin
7,8,POINT (456787.712 4974727.215),Linwood Beach,20700 Linwood Road,Excelsior,55331,None,HC Beach,Active,None,...,Outdoor,None,0,456787.7115,4.974727e+06,44.924662,-93.547556,11.712170,27053027400,Hennepin
8,9,POINT (452281.919 4975107.216),Lydiard Beach,Lydiard Avenue,Excelsior,55331,None,HC Beach,Active,None,...,Outdoor,None,0,452281.9191,4.975107e+06,44.927795,-93.604683,38.110868,27053027203,Hennepin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,308,POINT (474234.528 4969815.837),Southdale Shopping Center,6601 France Av S,Edina,55435,952-925-7874,Shopping Mall,Active,"Mon-Sat 10am-8pm, Sun 12pm-6pm",...,Indoor,None,74,474234.5278,4.969816e+06,44.881296,-93.326237,65.395251,27053024005,Hennepin
308,309,POINT (474134.345 4969357.928),Galleria,69th Street & France Ave,Edina,55435,952-925-4321,Shopping Mall,Active,"Mon-Sat 10am-7pm, Sun 11am-6pm",...,Indoor,None,75,474134.3458,4.969358e+06,44.877171,-93.327482,65.395251,27053024005,Hennepin
309,310,POINT (479421.895 4977265.478),Midtown Global Market,920 E. Lake St.,Minneapolis,55407,612-964-6932,Shopping Mall,Active,"Mon-Sat 10am-8pm, Sun 11am-6pm.",...,Indoor,Customers can park for free in the ramp. Indiv...,0,479421.8953,4.977265e+06,44.948527,-93.260859,70.764093,27053125800,Hennepin
311,312,POINT (465794.939 4967812.186),Asia Mall,12160 Technology Dr.,Eden Prairie,55344,None,Shopping Mall,Active,Open daily 10am-10pm.,...,Indoor,None,0,465794.9392,4.967812e+06,44.862905,-93.432959,58.738406,27053026019,Hennepin


In [49]:
#run check to see if points are inside hennepin
cooling_options_gdf["inside_hennepin"] = cooling_options_gdf.geometry.within(hennepin_county.geometry.iloc[0])

inside_count = cooling_options_gdf["inside_hennepin"].sum()
total_count = len(cooling_options_gdf)

print(f"{inside_count} out of {total_count} points are within Hennepin County.")

246 out of 246 points are within Hennepin County.


In [55]:
#replace existing cooling options table in database with the now filtered one

db_connection = "postgresql://postgres:''@34.30.71.239:5432/gis_data"
engine = create_engine(db_connection)

cooling_options_gdf.to_postgis(name="cooling_options", con=engine, if_exists="replace", index=False)
     